In [4]:
print("Om Vakraduntaaya Nama")

Om Vakraduntaaya Nama


In [2]:
! pip install mlxtend

     |████████████████████████████████| 1.3MB 2.7MB/s eta 0:00:01
     |████████████████████████████████| 10.1MB 49kB/s  eta 0:00:01
     |████████████████████████████████| 26.1MB 25kB/s  eta 0:00:01    |████                            | 3.2MB 7.8MB/s eta 0:00:03     |██████████████▎                 | 11.6MB 165kB/s eta 0:01:28/s eta 0:01:10█████████████████████████     | 22.0MB 2.7MB/s eta 0:00:02ta 0:00:01
ERROR: scikit-image 0.16.2 has requirement matplotlib!=3.0.0,>=2.0.0, but you'll have matplotlib 3.0.0 which is incompatible.
ERROR: azureml-train-automl 1.0.72 has requirement onnxmltools==1.4.1, but you'll have onnxmltools 1.6.0 which is incompatible.
ERROR: azureml-train-automl 1.0.72 has requirement pandas<=0.23.4,>=0.21.0, but you'll have pandas 1.0.4 which is incompatible.
ERROR: azureml-train-automl 1.0.72 has requirement scipy<=1.1.0,>=1.0.0, but you'll have scipy 1.4.1 which is incompatible.
ERROR: azureml-opendatasets 1.0.72.1 has requirement pandas<=0.23.4,>=0.21.0, but 

In [7]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [8]:
df = pd.read_excel("/home/nbuser/Retail Sales.xlsx")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df.shape


In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
# Stripping spaces in the description column
df['Description'] = df['Description'].str.strip()
# Dropping rows that doesn't contain involice numbers
df.dropna(axis = 0, subset=['InvoiceNo'], inplace = True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
# Remove credit transactions
df = df[~df['InvoiceNo'].str.contains('C')]

In [ ]:
df.groupby('Country').count().reset_index().sort_values('InvoiceNo', ascending = False).head()

* 1-hot encoding : This is the process pf consolidating items into one transaction per row.

* This can be done manually like below, or via the mlxtend.

* The one-hot encoding from mxltend encodes transaction data in form of a Python list into a NumPy integer array.

* The colums represent unique items present in the input array, and rows represent the individual transactions.

* Before proceeding with the 1-hot encoding, let us see the number of transactions by country

In [ ]:
Basket = (df[df['Country']=="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

Basket.head()

In [ ]:
def sum_to_boolean(x):
    if x<=0:
        return 0
    else:
        return 1

Basket_Final = Basket.applymap(sum_to_boolean)

In [ ]:

Basket_Final.drop('POSTAGE', inplace=True, axis=1)

Basket_Final.head()

In [ ]:
## Apriori to select the most important itemsets
Frequent_itemsets = apriori(Basket_Final, min_support = 0.06, use_colnames = True)

Frequent_itemsets.sort_values('support', ascending = False).head()

In [ ]:
Asso_Rules = association_rules(Frequent_itemsets, metric = "lift", min_threshold = 1)
Asso_Rules.sort_values('lift',ascending = False)

### Purchase behaviour in UK:

In [ ]:
Basket_France = (df[df['Country']=="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

def sum_to_boolean(x):
    if x<=0:
        return 0
    else:
        return 1

Basket_Final_France = Basket_France.applymap(sum_to_boolean)
Frequent_itemsets_France = apriori(Basket_Final_France, min_support = 0.06, use_colnames = True)

Asso_Rules_France = association_rules(Frequent_itemsets_France, metric = "lift", min_threshold = 1)
Asso_Rules_France.sort_values('lift',ascending = False).head()